我们将从零开始实现一个基于字符级循环神经网络的语言模型，并在周杰伦专辑歌词数据集上训练一个模型来进行歌词创作。首先，我们读取周杰伦专辑歌词数据集

In [6]:
import time
import math
import numpy as np
import torch
from torch import nn, optim
import torch.nn.functional as F
import sys
sys.path.append("..")
import d2lzh_torch as d2l
import zipfile
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

## 载入数据集

In [9]:
def load_data_jay_lyrics():
    """加载周杰伦歌词数据集"""
    with zipfile.ZipFile('./data/jaychou_lyrics.txt.zip') as zin:
        with zin.open('jaychou_lyrics.txt') as f:
            corpus_chars = f.read().decode('utf-8')
    corpus_chars = corpus_chars.replace('\n', ' ').replace('\r', ' ')
    corpus_chars = corpus_chars[0:10000]
    idx_to_char = list(set(corpus_chars))
    char_to_idx = dict([(char, i) for i, char in enumerate(idx_to_char)])
    vocab_size = len(char_to_idx)
    corpus_indices = [char_to_idx[char] for char in corpus_chars]
    return corpus_indices, char_to_idx, idx_to_char, vocab_size

In [10]:
(corpus_indices, char_to_idx, idx_to_char, vocab_size) = load_data_jay_lyrics()
vocab_size

1027

## one-hot向量
为了将词表示成向量输入到神经网络，一个简单的办法是使用one-hot向量。

假设词典中不同字符的数量为NN（即词典大小vocab_size），每个字符已经同一个从0到N−1N−1的连续整数值索引一一对应。

如果一个字符的索引是整数ii, 那么我们创建一个全0的长为NN的向量，并将其位置为ii的元素设成1。该向量就是对原字符的one-hot向量。

下面分别展示了索引为0和2的one-hot向量，向量长度等于词典大小。

In [ ]:
def to_oneshot(X,n_class):
    
    